In [1]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [2]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['punc_rumor'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['punc_truth'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# jieba分词
import jieba
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/jieba.cache
Loading model cost 0.448 seconds.
Prefix dict has been built successfully.
/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_21642/2493417845.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


#### 隐形狄利克雷分布原理
- 设想一台自动生成文档的机器，只有两个选项来控制生成文档的两个属性
    - 生成文档的词的数量（泊松分布）
    - 文档中混合的主题的数量（狄利克雷分布）
- 除此之外，最关键的问题在于确定一个主题-词项矩阵，该矩阵表示了每个词对主题的贡献权重。一但能够确定这个矩阵，机器就可以在选择好的主题上反复迭代选择词，直到生成一篇足够长度的文档。
- 回到对现有文档估计主题的问题上来，LDiA可以用于关于词和主题的参数，Blei和Ng通过分析语料库中文档的统计数据确定两个参数
    - 第一个参数可以通过计算语料库的平均词（n-gram）数量、
    - 第二个参数更棘手，必须先猜测有几个主题数再为主题分配语词，最后优化目标函数
-  “LSA试图将原本分散的东西分散开，LDiA试图将原本接近的东西接近在一起”

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
try:
    print('rumor0:'+df.loc['rumor0'].text)
    print(bow_docs.loc['rumor0'][bow_docs.loc['rumor0'] > 0].head())
except KeyError:
    print('rumor0!:'+df.loc['rumor0!'].text)
    print(bow_docs.loc['rumor0!'][bow_docs.loc['rumor0!'] > 0].head())

rumor0:大家 都 胸闷 憋气 不 舒服 都 觉得 心脏 问题 不是 说 胸闷 憋气 检查 都 正常 行 胸口 疼 都 心脏 关系 游戏 中 女性 年轻 女性 很 可能 精神压力 大 医生 说 心脏 没事儿
不     1
不是    1
中     1
关系    1
医生    1
Name: rumor0, dtype: int64


In [5]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)  # <1>
ldia.components_.shape

(16, 9200)

In [6]:
columns = ['topic{}'.format(i) for i in range(ldia.components_.shape[0])]
pd.set_option('display.width', 2)
componets = pd.DataFrame(ldia.components_.T,index = terms ,columns=columns)
componets.round(2).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
%,0.06,0.06,0.06,1.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
.,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,1.06,2.06,0.06,0.06
0,0.06,0.06,0.06,1.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06


In [7]:
componets.topic0.sort_values(axis=0, ascending=False).head(10)

风险     126.779308
会      104.137411
降低      66.998038
发生      66.390340
糖尿病     52.532224
发病      44.188418
增加      41.142041
引起      38.931123
幽门      37.115259
研究      31.756433
Name: topic0, dtype: float64

In [8]:
ldia16_topic_vectors = ldia.transform(bow_docs)
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors,
                                    index=index, columns=columns)
ldia16_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
rumor0,0.00,0.00,0.00,0.00,0.26,0.00,0.00,0.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
rumor1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.97,0.00,0.00,0.00,0.00
rumor2!,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.84,0.01,0.01,0.01,0.01,0.01,0.01,0.01
rumor3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.97,0.00,0.00,0.00,0.00
rumor4,0.00,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.91,0.00,0.00


#### 使用LDiA主题向量训练LDA模型

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(ldia16_topic_vectors,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['ldia16_rumor'] = lda.predict(ldia16_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1479
过采样前训练集中类别1的数量： 748
过采样后训练集中类别0的数量： 1479
过采样后训练集中类别1的数量： 1479


0.596

In [10]:
print(y_test.index)

Index(['rumor507!',
       'rumor2268',
       'rumor2349',
       'rumor1788',
       'rumor949',
       'rumor324',
       'rumor1090',
       'rumor1463!',
       'rumor2227',
       'rumor420',
       ...
       'rumor1309',
       'rumor2408',
       'rumor2226!',
       'rumor461!',
       'rumor927!',
       'rumor157',
       'rumor1657',
       'rumor1724',
       'rumor238!',
       'rumor1813!'],
      dtype='object', length=557)


In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].ldia16_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["非rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)

              precision    recall  f1-score   support

      非rumor     0.7720    0.6046    0.6781       392
       rumor     0.3800    0.5758    0.4578       165

    accuracy                         0.5961       557
   macro avg     0.5760    0.5902    0.5680       557
weighted avg     0.6559    0.5961    0.6129       557

[[237 155]
 [ 70  95]]


#### 对比TF-IDF向量的分类

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize,)
tfidf_docs =tfidf.fit_transform(raw_documents=df.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(tfidf_docs,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['tfidf_rumor'] = lda.predict(tfidf_docs)

过采样前训练集中类别0的数量： 1479
过采样前训练集中类别1的数量： 748
过采样后训练集中类别0的数量： 1479
过采样后训练集中类别1的数量： 1479


In [14]:
round(float(lda.score(X_test, y_test)), 3)

0.786

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].tfidf_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)


              precision    recall  f1-score   support

   non-rumor     0.8934    0.7908    0.8390       392
       rumor     0.6095    0.7758    0.6827       165

    accuracy                         0.7864       557
   macro avg     0.7514    0.7833    0.7608       557
weighted avg     0.8093    0.7864    0.7927       557

[[310  82]
 [ 37 128]]


#### LDiA使用32个主题

In [16]:
ldia32 = LDiA(n_components=300, learning_method='batch')
ldia32 = ldia32.fit(bow_docs)  # <1>
weights = pd.DataFrame(ldia32.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(ldia32.components_.shape[0])])
ldia32.components_.shape

(300, 9200)

In [17]:
ldia32_topic_vectors = ldia32.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia32.components_.shape[0])]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors,index=index, columns=columns32)
ldia32_topic_vectors.round(4).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic290,topic291,topic292,topic293,topic294,topic295,topic296,topic297,topic298,topic299
rumor0,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
rumor1,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
rumor2!,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,...,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006
rumor3,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
rumor4,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(ldia32_topic_vectors,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['ldia32_rumor'] = lda.predict(ldia32_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1479
过采样前训练集中类别1的数量： 748
过采样后训练集中类别0的数量： 1479
过采样后训练集中类别1的数量： 1479


0.882

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].ldia32_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)


              precision    recall  f1-score   support

   non-rumor     0.9075    0.9260    0.9167       392
       rumor     0.8153    0.7758    0.7950       165

    accuracy                         0.8815       557
   macro avg     0.8614    0.8509    0.8558       557
weighted avg     0.8802    0.8815    0.8806       557

[[363  29]
 [ 37 128]]
